In [ ]:
"""
Stuff LCEL Chain.


execute and after langchain.smith view.
"""

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
# from langchain.llms import OpenLLM
# from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/") # gitignore 추가.

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# vectorstore = Chroma.from_documents(docs, cache_embeddings)
vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages({
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
})

# 검색 방법 설정.
chain = {"context": retriever, "question": RunnablePassthrough() } | prompt | llm

chain.invoke("Describe Victory Mansions.")

AIMessage(content="Victory Mansions is an old building with flats that were built around 1930. The flats are falling apart, with constant flaking plaster, burst pipes, leaky roofs, and a heating system that is often not working properly due to cost-saving measures. The building has a general smell of boiled cabbage mixed with sweat. The Parsons' flat, which is bigger than Winston's, has a battered and trampled-on look, with sports equipment scattered around, dirty dishes, and posters of Big Brother on the walls.")